# Temporal file indexer

### I wanted a quick way to figure out "when" sequential files were missing.
### As a result, I created a pythonic way to index files via their timestamp, and do some interesting datetime subsetting.

In [1]:
import pandas
import numpy
import datetime
import pickle

pandas.date_range is a good way to generate time stamps without relying on the time stamps provided by the data.

Data timestamps can be irregular, but with date_range you can supply equal intervals.

In this case, freq='60T' tells date_range to create datetimes every 60 minutes from start to end

In [2]:
datetimeRange = pandas.date_range(start=datetime.datetime(2015,11,1,0,0),end=datetime.datetime(2015,11,1,12,0),freq='60T')

df = pandas.DataFrame(datetimeRange,columns=["time"])

df

,time
0,2015-11-01 00:00:00
1,2015-11-01 01:00:00
2,2015-11-01 02:00:00
3,2015-11-01 03:00:00
4,2015-11-01 04:00:00
5,2015-11-01 05:00:00
6,2015-11-01 06:00:00
7,2015-11-01 07:00:00
8,2015-11-01 08:00:00
9,2015-11-01 09:00:00


What is useful is you can now join existing data, with associated time stamps, to your defined interval DataFrame.

Here, I generate some random data with associated timestamps.

In [3]:
sampleDataRange = pandas.date_range(datetime.datetime(2015,11,1,0,0),datetime.datetime(2015,11,1,15,0),freq='60T')

data = numpy.array(list(zip(sampleDataRange,numpy.random.randint(0,100,(15)),numpy.random.randint(0,100,(15)))))

df2 = pandas.DataFrame(data,columns=["time","data1","data2"])

df2

,time,data1,data2
0,2015-11-01 00:00:00,3,77
1,2015-11-01 01:00:00,43,63
2,2015-11-01 02:00:00,91,1
3,2015-11-01 03:00:00,23,16
4,2015-11-01 04:00:00,52,58
5,2015-11-01 05:00:00,28,60
6,2015-11-01 06:00:00,0,99
7,2015-11-01 07:00:00,1,90
8,2015-11-01 08:00:00,37,10
9,2015-11-01 09:00:00,87,78


In this case, I want to eventually do datetime subsetting, so I set the index to the timestamp.  

By default, the join is on the index field.

Notice the joined DataFrame (df2), has data until 14:00, whereas df1 only has data until 12:00

Since I chose left join, the calling DataFrame determines what data is included (opposite for right join).

In [4]:
df_ = df.set_index('time')
df2_ = df2.set_index('time')

df3 = df_.join(df2_,how='left')

df3

,data1,data2
time,,
2015-11-01 00:00:00,3,77
2015-11-01 01:00:00,43,63
2015-11-01 02:00:00,91,1
2015-11-01 03:00:00,23,16
2015-11-01 04:00:00,52,58
2015-11-01 05:00:00,28,60
2015-11-01 06:00:00,0,99
2015-11-01 07:00:00,1,90
2015-11-01 08:00:00,37,10


For files, its nice to know where data is missing.

This is done naturally through a join, where NaN means your files are missing for that timestamp.

I just generated random, file-location-looking, strings to build a DataFrame

In [5]:
dateRangeData = pandas.date_range(datetime.datetime(2015,11,1,6,0),datetime.datetime(2015,11,1,18,0),freq='60T')

prefix = "C:\\Users\\SampleUser\\DataFiles\\"

suffix = ".dat"

fileNames = numpy.random.randint(50,500,(12))

fileLocations = [prefix+str(fn).zfill(3)+suffix for fn in fileNames]

tupleData = numpy.array(list(zip(dateRangeData,fileLocations)))

df4 = pandas.DataFrame(tupleData,columns=["time","filename"])

df4 = df4.set_index('time')

df5 = df_.join(df4,how='left')

#df4
df5

,filename
time,
2015-11-01 00:00:00,NaN
2015-11-01 01:00:00,NaN
2015-11-01 02:00:00,NaN
2015-11-01 03:00:00,NaN
2015-11-01 04:00:00,NaN
2015-11-01 05:00:00,NaN
2015-11-01 06:00:00,C:\Users\SampleUser\DataFiles\329.dat
2015-11-01 07:00:00,C:\Users\SampleUser\DataFiles\379.dat
2015-11-01 08:00:00,C:\Users\SampleUser\DataFiles\212.dat


To remove all timestamps with missing files, you can use dropna()

In [6]:
df6 = df5.dropna()

df6

,filename
time,
2015-11-01 06:00:00,C:\Users\SampleUser\DataFiles\329.dat
2015-11-01 07:00:00,C:\Users\SampleUser\DataFiles\379.dat
2015-11-01 08:00:00,C:\Users\SampleUser\DataFiles\212.dat
2015-11-01 09:00:00,C:\Users\SampleUser\DataFiles\394.dat
2015-11-01 10:00:00,C:\Users\SampleUser\DataFiles\132.dat
2015-11-01 11:00:00,C:\Users\SampleUser\DataFiles\493.dat
2015-11-01 12:00:00,C:\Users\SampleUser\DataFiles\371.dat


You can then serialize this pandas object with pickle.

In this trivial example, it is kind of pointless to do this.  But if you have millions of files, this can be useful.

Save the pandas object by pickle.dump, load it with pickle.load.  The open(..) is neccesary to include.

Finding null files is used with a combination of isnull()

In [7]:
pickle.dump(df5,open('indexed_files.pkl','wb'))

files = pickle.load(open('indexed_files.pkl','rb'))

files_ = files.dropna()

print("Matched filenames")
print(" ")
print(files_)

files_ = files[files['filename'].isnull()]

print("Unmatched times with filenames")
print(" ")
print(files_)

Matched filenames
 
                                                  filename
time                                                      
2015-11-01 06:00:00  C:\Users\SampleUser\DataFiles\329.dat
2015-11-01 07:00:00  C:\Users\SampleUser\DataFiles\379.dat
2015-11-01 08:00:00  C:\Users\SampleUser\DataFiles\212.dat
2015-11-01 09:00:00  C:\Users\SampleUser\DataFiles\394.dat
2015-11-01 10:00:00  C:\Users\SampleUser\DataFiles\132.dat
2015-11-01 11:00:00  C:\Users\SampleUser\DataFiles\493.dat
2015-11-01 12:00:00  C:\Users\SampleUser\DataFiles\371.dat
Unmatched times with filenames
 
                    filename
time                        
2015-11-01 00:00:00      NaN
2015-11-01 01:00:00      NaN
2015-11-01 02:00:00      NaN
2015-11-01 03:00:00      NaN
2015-11-01 04:00:00      NaN
2015-11-01 05:00:00      NaN


Heres a look at whats going on during files[files['filename'].isnull()]

In [8]:
tf = ["yay! a file",None]

a = numpy.random.choice(tf,size=(12))
b = numpy.random.choice(tf,size=(12))
c = numpy.random.choice(tf,size=(12))
d = numpy.random.choice(tf,size=(12))

tupleData = numpy.array(list(zip(dateRangeData,a,b,c,d)))

df7 = pandas.DataFrame(tupleData,columns=['time','a','b','c','d'])
df7 = df7.set_index('time')

print(" ")
print(df7)

print(" ")
print("Where are NaNs??")
print(df7.isnull())

print(" ")
print("Drop all NaNs")
print(df7.dropna())

print(" ")
print("Drop where a is NaN")
print(df7[df7['a'].isnull()])

print(" ")
print("Drop where b is NaN")
print(df7[df7['b'].isnull()])

print(" ")
print("Drop where c is NaN")
print(df7[df7['c'].isnull()])

print(" ")
print("Drop where d is NaN")
print(df7[df7['c'].isnull()])

 
                               a            b            c            d
time                                                                   
2015-11-01 06:00:00  yay! a file         None  yay! a file  yay! a file
2015-11-01 07:00:00         None  yay! a file         None         None
2015-11-01 08:00:00  yay! a file  yay! a file         None         None
2015-11-01 09:00:00  yay! a file  yay! a file  yay! a file  yay! a file
2015-11-01 10:00:00         None  yay! a file         None  yay! a file
2015-11-01 11:00:00         None  yay! a file         None         None
2015-11-01 12:00:00         None         None         None         None
2015-11-01 13:00:00         None  yay! a file         None         None
2015-11-01 14:00:00         None  yay! a file         None  yay! a file
2015-11-01 15:00:00         None  yay! a file         None         None
2015-11-01 16:00:00         None         None  yay! a file  yay! a file
2015-11-01 17:00:00         None         None  yay! a file    

OK.. makes sense

And now kind of to the point of why I did this--datetime subsetting.

In [9]:
files['2015-11-01 00:00:00':'2015-11-01 04:00:00']

,filename
time,
2015-11-01 00:00:00,NaN
2015-11-01 01:00:00,NaN
2015-11-01 02:00:00,NaN
2015-11-01 03:00:00,NaN
2015-11-01 04:00:00,NaN


In [10]:
files['2015-11-01 06:00:00':'2015-11-01 12:00:00']

,filename
time,
2015-11-01 06:00:00,C:\Users\SampleUser\DataFiles\329.dat
2015-11-01 07:00:00,C:\Users\SampleUser\DataFiles\379.dat
2015-11-01 08:00:00,C:\Users\SampleUser\DataFiles\212.dat
2015-11-01 09:00:00,C:\Users\SampleUser\DataFiles\394.dat
2015-11-01 10:00:00,C:\Users\SampleUser\DataFiles\132.dat
2015-11-01 11:00:00,C:\Users\SampleUser\DataFiles\493.dat
2015-11-01 12:00:00,C:\Users\SampleUser\DataFiles\371.dat


So now, you don't have to call glob, os.dir, etc., and you can subset your file locations by time.

Ultimately, you would then read in these files and do analyses on particular time periods.